In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import layers

In [3]:
# turn list of lists of equivalent braids with corresponding answers into trainable data

def parse_equivalent_braids(allBraids, answers, maxCrossings):

    x_data = []
    y_data = []

    for equivalentBraids, answer in zip(allBraids, answers):
        for braid in tf.keras.preprocessing.sequence.pad_sequences(
            equivalentBraids, padding="post", maxlen=maxCrossings):
            x_data.append(braid)
            y_data.append(1 if answer=="Y" else 0)
        
    return np.asarray(x_data), np.asarray(y_data)

In [10]:
# read pre-parsed knot data with braid words and equivalent braids

knot_data = pd.read_csv('../data/knot_data.csv')
knot_data["Equivalent Braids"] = knot_data["Equivalent Braids"].apply(eval)

In [75]:
maxCrossings = 14
maxStrands = 6

In [89]:
# split dataset for training

train_split = .8

train_data = knot_data.sample(frac=train_split)
test_data = knot_data.drop(train_data.index)

x_train, y_train = parse_equivalent_braids(train_data["Equivalent Braids"],
                    train_data["Q-Positive"], maxCrossings)

x_train = tf.keras.utils.to_categorical(x_train, num_classes=maxStrands*2+1)

x_test, y_test = parse_equivalent_braids(test_data["Equivalent Braids"],
                    test_data["Q-Positive"], maxCrossings)

x_test = tf.keras.utils.to_categorical(x_test, num_classes=maxStrands*2+1)


In [90]:
# build model from functional api

inputs = keras.Input(shape=(maxCrossings, maxStrands*2+1))

x = layers.Flatten()(inputs)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(64, activation="relu")(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

In [91]:
model = keras.Model(inputs=inputs, outputs=outputs, 
    name="quasipositivity_functional")

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [92]:
model.summary()

Model: "quasipositivity_functional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 14, 13)]          0         
                                                                 
 flatten_5 (Flatten)         (None, 182)               0         
                                                                 
 dense_15 (Dense)            (None, 64)                11712     
                                                                 
 dense_16 (Dense)            (None, 64)                4160      
                                                                 
 dense_17 (Dense)            (None, 1)                 65        
                                                                 
Total params: 15,937
Trainable params: 15,937
Non-trainable params: 0
_________________________________________________________________


In [93]:
history = model.fit(
    x_train,
    y_train,
    batch_size=16,
    epochs=5
)

Epoch 1/5
398/398 [==============================] - 0s 474us/step - loss: 0.4562 - accuracy: 0.8042
Epoch 2/5
398/398 [==============================] - 0s 465us/step - loss: 0.2951 - accuracy: 0.8891
Epoch 3/5
398/398 [==============================] - 0s 464us/step - loss: 0.1965 - accuracy: 0.9260
Epoch 4/5
398/398 [==============================] - 0s 447us/step - loss: 0.1406 - accuracy: 0.9474
Epoch 5/5
398/398 [==============================] - 0s 453us/step - loss: 0.1032 - accuracy: 0.9615


In [94]:
model.evaluate(x_test, y_test)

50/50 [==============================] - 0s 412us/step - loss: 0.2605 - accuracy: 0.9181


[0.2605123817920685, 0.9181249737739563]